# Appendix B

In [ ]:
%run header.py

Load the default opacities for comparison and calculate the size-distribution-averages

In [ ]:
d      = np.load(opacity.get_datafile('default_opacities_smooth.npz'))
a      = d['a']
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']
gsca   = d['g']

k_sca_eff = (1 - gsca) * k_sca
eps_nu = k_abs / (k_abs + k_sca_eff)

lam_avg = [0.1, 0.3]
q       = [3.5]
res     = [opacity.size_average_opacity(lam_avg, a, lam, k_abs, k_sca, q=_q, plot=False) for _q in q]
res_eff = [opacity.size_average_opacity(lam_avg, a, lam, k_abs, k_sca_eff, q=_q, plot=False) for _q in q]

eps_eff = [_res_eff['ka'] / (_res_eff['ka'] + _res_eff['ks']) for _res_eff in res_eff]

Create the opacities from scratch, varying the water or vacuum fraction

In [ ]:
density_water     = 0.92
density_silicates = 3.30
density_troilite  = 4.83
density_organics  = 1.50

# default values

constants_default = [
    opacity.diel_warrenbrandt08(),
    opacity.diel_draine2003('astrosilicates'),
    opacity.diel_henning('troilite'),
    opacity.diel_henning('organics'),
    ]

densities_default = np.array([
    density_water,
    density_silicates,
    density_troilite,
    density_organics,
    ])

fv_default = np.array([
    0.3642,
    0.1670,
    0.0258,
    0.4430])

# porous values
porosity = 0.8
constants_porous = [opacity.diel_vacuum()] + constants_default
densities_porous = np.hstack((0.0, densities_default))
fv_porous = np.hstack((porosity, (1 - porosity) * fv_default))

# different water fraction
water_volfrac = 0.6
constants_water = constants_default
densities_water = densities_default
fact = fv_default[0]/water_volfrac
fv_water = np.hstack((water_volfrac, fv_default[1:]/fv_default[1:].sum() * (1 - water_volfrac)))


# now different carbon species:
# 01: Jaeger 98

constants_c01 = constants_default.copy()
densities_c01 = densities_default.copy()
d = opacity.diel_jaeger98(800)
d.extrapolate_constants_up(lmin=.04, lmax=10,kind='linear')
d.extrapolate_constants_down(lmin=1e-5, lmax=5e-5,kind='linear')
constants_c01[-1] = d
densities_c01[-1] = 1.843
fv_c01            = fv_default.copy()

# 02: Zubko

constants_c02     = constants_default.copy()
densities_c02     = densities_default.copy()
constants_c02[-1] = opacity.diel_zubko96(extrapol=True, lmax=10.0, sample='BE')
densities_c02[-1] = 2.5
fv_c02            = fv_default.copy()

# 03: Draine: 0.01 parallel

constants_c03     = constants_default.copy()
densities_c03     = densities_default.copy()
constants_c03[-1] = opacity.diel_draine2003('graphite', parallel=True, a='0.01')
densities_c03[-1] = 2.26
fv_c03            = fv_default.copy()

# 04: Draine: 0.01 perpendicular

constants_c04     = constants_default.copy()
densities_c04     = densities_default.copy()
constants_c04[-1] = opacity.diel_draine2003('graphite', parallel=False, a='0.01')
densities_c04[-1] = 2.26
fv_c04            = fv_default.copy()

# 05: Draine: 0.10 perpendicular

constants_c05     = constants_default.copy()
densities_c05     = densities_default.copy()
constants_c05[-1] = opacity.diel_draine2003('graphite', parallel=False, a='0.10')
densities_c05[-1] = 2.26
fv_c05            = fv_default.copy()

# 06: Draine: 0.10 perpendicular

constants_c06     = constants_default.copy()
densities_c06     = densities_default.copy()
constants_c06[-1] = opacity.diel_draine2003('graphite', parallel=False, a='0.10')
densities_c06[-1] = 2.26
fv_c06            = fv_default.copy()

# mix densities
rhos_default = (fv_default * densities_default).sum()
rhos_porous  = (fv_porous * densities_porous).sum()
rhos_water   = (fv_water * densities_water).sum()
rhos_c01     = (fv_c01 * densities_c01).sum()
rhos_c02     = (fv_c02 * densities_c02).sum()
rhos_c03     = (fv_c03 * densities_c03).sum()
rhos_c04     = (fv_c04 * densities_c04).sum()
rhos_c05     = (fv_c05 * densities_c05).sum()
rhos_c06     = (fv_c06 * densities_c06).sum()

In [ ]:
# get the mixes of optical constants using the Bruggeman rule

d_def = opacity.diel_mixed(constants_default, fv_default, rule='Bruggeman')
d_por = opacity.diel_mixed(constants_porous,  fv_porous,  rule='Maxwell-Garnett')
d_h2o = opacity.diel_mixed(constants_water,   fv_water,   rule='Bruggeman')
d_c01 = opacity.diel_mixed(constants_c01,     fv_c01,     rule='Bruggeman')
d_c02 = opacity.diel_mixed(constants_c02,     fv_c02,     rule='Bruggeman')
#d_c03 = opacity.diel_mixed(constants_c03,     fv_c03,     rule='Bruggeman')
#d_c04 = opacity.diel_mixed(constants_c04,     fv_c04,     rule='Bruggeman')
#d_c05 = opacity.diel_mixed(constants_c05,     fv_c05,     rule='Bruggeman')
d_c06 = opacity.diel_mixed(constants_c06,     fv_c06,     rule='Bruggeman')

# Calculate the Mie opacities, this takes *very* long if `extrapolate_large_grains=False`.

res_def = opacity.get_opacities(a, lam, rho_s=rhos_default, diel_const=d_def, return_all=True, extrapol=True, extrapolate_large_grains=True)
res_por = opacity.get_opacities(a, lam, rho_s=rhos_porous,  diel_const=d_por, return_all=True, extrapol=True, extrapolate_large_grains=True)
res_h2o = opacity.get_opacities(a, lam, rho_s=rhos_water,   diel_const=d_h2o, return_all=True, extrapol=True, extrapolate_large_grains=True)
res_c01 = opacity.get_opacities(a, lam, rho_s=rhos_c01,     diel_const=d_c01, return_all=True, extrapol=True, extrapolate_large_grains=True)
res_c02 = opacity.get_opacities(a, lam, rho_s=rhos_c02,     diel_const=d_c02, return_all=True, extrapol=True, extrapolate_large_grains=True)
#res_c03 = opacity.get_opacities(a, lam, rho_s=rhos_c03,     diel_const=d_c03, return_all=True, extrapol=True, extrapolate_large_grains=True)
#res_c04 = opacity.get_opacities(a, lam, rho_s=rhos_c04,     diel_const=d_c04, return_all=True, extrapol=True, extrapolate_large_grains=True)
#res_c05 = opacity.get_opacities(a, lam, rho_s=rhos_c05,     diel_const=d_c05, return_all=True, extrapol=True, extrapolate_large_grains=True)
res_c06 = opacity.get_opacities(a, lam, rho_s=rhos_c06,     diel_const=d_c06, return_all=True, extrapol=True, extrapolate_large_grains=True)

Size averaging

In [ ]:
lam_avg =[0.1, 0.3]

r_def = opacity.size_average_opacity(lam_avg, a, lam, res_def['k_abs'], res_def['k_sca'], plot=False)
r_por = opacity.size_average_opacity(lam_avg, a, lam, res_por['k_abs'], res_por['k_sca'], plot=False)
r_h2o = opacity.size_average_opacity(lam_avg, a, lam, res_h2o['k_abs'], res_h2o['k_sca'], plot=False)
r_c01 = opacity.size_average_opacity(lam_avg, a, lam, res_c01['k_abs'], res_c01['k_sca'], plot=False)
r_c02 = opacity.size_average_opacity(lam_avg, a, lam, res_c02['k_abs'], res_c02['k_sca'], plot=False)
#r_c03 = opacity.size_average_opacity(lam_avg, a, lam, res_c03['k_abs'], res_c03['k_sca'], plot=False)
#r_c04 = opacity.size_average_opacity(lam_avg, a, lam, res_c04['k_abs'], res_c04['k_sca'], plot=False)
#r_c05 = opacity.size_average_opacity(lam_avg, a, lam, res_c05['k_abs'], res_c05['k_sca'], plot=False)
r_c06 = opacity.size_average_opacity(lam_avg, a, lam, res_c06['k_abs'], res_c06['k_sca'], plot=False)

In [ ]:
# the effective ones to calculate epsilon

re_def = opacity.size_average_opacity(lam_avg, a, lam, res_def['k_abs'], (1 - res_def['g']) * res_def['k_sca'], plot=False)
re_por = opacity.size_average_opacity(lam_avg, a, lam, res_por['k_abs'], (1 - res_por['g']) * res_por['k_sca'], plot=False)
re_h2o = opacity.size_average_opacity(lam_avg, a, lam, res_h2o['k_abs'], (1 - res_h2o['g']) * res_h2o['k_sca'], plot=False)
re_c01 = opacity.size_average_opacity(lam_avg, a, lam, res_c01['k_abs'], (1 - res_c01['g']) * res_c01['k_sca'], plot=False)
re_c02 = opacity.size_average_opacity(lam_avg, a, lam, res_c02['k_abs'], (1 - res_c02['g']) * res_c02['k_sca'], plot=False)
#re_c03 = opacity.size_average_opacity(lam_avg, a, lam, res_c03['k_abs'], (1 - res_c03['g']) * res_c03['k_sca'], plot=False)
#re_c04 = opacity.size_average_opacity(lam_avg, a, lam, res_c04['k_abs'], (1 - res_c04['g']) * res_c04['k_sca'], plot=False)
#re_c05 = opacity.size_average_opacity(lam_avg, a, lam, res_c05['k_abs'], (1 - res_c05['g']) * res_c05['k_sca'], plot=False)
re_c06 = opacity.size_average_opacity(lam_avg, a, lam, res_c06['k_abs'], (1 - res_c06['g']) * res_c06['k_sca'], plot=False)

eps_def = re_def['ka']/(re_def['ka'] + re_def['ks'])
eps_por = re_por['ka']/(re_por['ka'] + re_por['ks'])
eps_h2o = re_h2o['ka']/(re_h2o['ka'] + re_h2o['ks'])
eps_c01 = re_c01['ka']/(re_c01['ka'] + re_c01['ks'])
eps_c02 = re_c02['ka']/(re_c02['ka'] + re_c02['ks'])
#eps_c03 = re_c03['ka']/(re_c03['ka'] + re_c03['ks'])
#eps_c04 = re_c04['ka']/(re_c04['ka'] + re_c04['ks'])
#eps_c05 = re_c05['ka']/(re_c05['ka'] + re_c05['ks'])
eps_c06 = re_c06['ka']/(re_c06['ka'] + re_c06['ks'])

Plotting

In [ ]:
f, axs = plt.subplots(3, 1, figsize=(3.5, 3.5 * 3 * 0.60), sharex=True)
i_lam = 0
lines = []

# names     = ['default', 'porous', 'high-water', 'Jäger', 'Zubko', 'Draine #1', 'Draine #2', 'Draine #3', 'Draine #4']
# r_array   = [r_def,     r_por,    r_h2o,        r_c01,   r_c02,   r_c03,       r_c04,       r_c05,        r_c06]
# res_array = [res_def,   res_por,  res_h2o,      res_c01, res_c02, res_c03,     res_c04,     res_c05,      res_c06]
# eps_array = [eps_def,   eps_por,  eps_h2o,      eps_c01, eps_c02, eps_c03,     eps_c04,     eps_c05,      eps_c06]

# names     = ['default', 'porous', 'high-water', 'Jäger', 'Zubko', r'Draine ($a=0.01, \parallel$)', r'Draine ($a=0.10, \bot$)']
# r_array   = [r_def,     r_por,    r_h2o,        r_c01,   r_c02,   r_c03,                           r_c06]
# res_array = [res_def,   res_por,  res_h2o,      res_c01, res_c02, res_c03,                         res_c06]
# eps_array = [eps_def,   eps_por,  eps_h2o,      eps_c01, eps_c02, eps_c03,                         eps_c06]

names     = ['default', 'porous', 'high-water', 'Jäger', 'Zubko', r'Draine ($a=0.10, \bot$)']
r_array   = [r_def,     r_por,    r_h2o,        r_c01,   r_c02,   r_c06]
res_array = [res_def,   res_por,  res_h2o,      res_c01, res_c02, res_c06]
eps_array = [eps_def,   eps_por,  eps_h2o,      eps_c01, eps_c02, eps_c06]

# add dummy lines for the legend
    
lines += axs[0].plot([], [], 'k-', label='absoprtion')
lines += axs[0].plot([], [], 'k--', label='scattering')

for _res, name,_eps in zip(r_array, names, eps_array):
    ind   = names.index(name)
    lw    = 1 + (ind < 3)
    alpha = 1 - 0.25 * (ind > 2)
    # absorption & scattering opacity
    
    lines += axs[0].loglog(a, _res['ka'][i_lam], '-', label='{}'.format(name), lw=lw, alpha=alpha)
    
    # fix color of carbon species
    if ind > 2:
        color = (ind - 1) / (len(names) - 1) * np.ones(3)
        lines[-1].set_color(color)
    
    axs[0].loglog(         a, _res['ks'][i_lam], '--', c=lines[-1].get_color(), lw=lw, alpha=alpha)
    
    # spectral index
    
    axs[1].plot(a, _res['beta'], '-', label=name, c=lines[-1].get_color(), lw=lw, alpha=alpha)
    
    # epsilon
    
    axs[2].plot(a, _eps[i_lam,:], '-', label=name, c=lines[-1].get_color(), lw=lw, alpha=alpha)
    

# labels
    
axs[0].set_ylabel('$\kappa_\mathrm{{{:.0f}\,mm}}$ [cm$^2$/g]'.format(10 * lam_avg[i_lam]))
axs[1].set_ylabel(r'$\beta_\mathrm{{{:.0f}-{:.0f}\,mm}}$'.format(10 * lam_avg[0], 10 * lam_avg[1]))
axs[2].set_ylabel(r'$\epsilon^\mathrm{{eff}}_{{{:.0f}\,\mathrm{{mm}}}}$'.format(10 * lam_avg[0]))
axs[2].set_xlabel(r'$a_\mathrm{max}$ [cm]')

# legends
    
axs[0].legend(lines, [l.get_label() for l in lines], fontsize='x-small', loc='best', ncol=3)
axs[1].legend(fontsize='x-small', loc='best', ncol=2)
axs[2].legend(fontsize='x-small', loc='best', ncol=2)

# limits & layout

axs[0].set_ylim(1e-2, 1e2)
axs[1].set_ylim(0, 4)
axs[1].set_xlim(1e-4, 1e2)
axs[2].set_ylim(0, 1.1)

f.subplots_adjust(
    left=0.13, right=0.97,
    bottom=0.09, top=0.98,
    wspace=0.1, hspace=0.05)

f.savefig('figures/appendix_fig1.pdf')

Jäger & Draine $(a=0.01, \parallel)$ are similar and we could drop this Draine opacity

Plot the Henyey-Greenstein asymmetry factor $g$

In [ ]:
il = lam.searchsorted(.1)

f, ax = plt.subplots()

for name, g in zip(names, res_array):
    ax.semilogx(a, g['g'][:, il],label=name)

ax.set_xlabel('particle size [cm]')
ax.set_ylabel('asymmetry factor $g$')
#ax.set_ylim([1e-4,2])
ax.set_xlim([1e-5, 1e1])
ax.legend(fontsize='xx-small')

## Plot the averaged opacity as fct of wavelength

In [ ]:
# get the other opacity laws

kb = 3.446 * (lam / 0.087)**(-1)  # Beckwith 1990

# the opacities from Andrews et al. 2009

la, ka = np.loadtxt(opacity.get_datafile('andrews2009.dat')).T

In [ ]:
# make a size distribution up to 1 mm

s        = a**0.5
ia       = np.abs(a - 0.1).argmin()
s[ia+1:] = 0.
s        = s / s.sum()

print('using amax = {} mm'.format(a[np.where(s > 0)[0][-1]] * 10))

f, ax = plt.subplots(figsize=(7, 7 / 1.68 * 1.039))

for res, name in zip(res_array, names):

    # average the opacities with that size distribution

    k_a_avg = (res['k_abs'].T * s).sum(1)

    ax.loglog(lam, k_a_avg, label=name)
    
# other opacities
ax.loglog(la * 1e-4, ka, 'k--', zorder=-100, label='Andrews et al. 2009')
ax.loglog(lam,       kb, 'k--', zorder=-100, alpha=0.5, label='Beckwith et al. 1990')
    
# format the plot
    
ax.legend(fontsize='small', loc=3, ncol=2)

ax.set_xlim(1e-5, 1e0)
ax.set_ylim(1e-2, 2e3)

ax.set_xlabel(r'$\lambda$ [cm]')
ax.set_ylabel(r'$\kappa_\nu^\mathrm{abs,tot}$ [cm$^2$/g]')

f.subplots_adjust(
    left=0.09, right=0.97,
    bottom=0.1, top=0.98,
    wspace=0.1, hspace=0.05)

f.savefig('figures/appendix_fig2.pdf')